In [1]:
%cd ../..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

[striatum:1061072] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.striatum.1000/jf.0/1297940480/shared_mem_cuda_pool.striatum could be created.
[striatum:1061072] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [7]:
NWB_SUBJECTS = glob.glob('/mnt/data/000253/sub-*/')

In [8]:
PILOT_FILES = []

In [9]:
def hippocampal_channels(channels):
    for l in channels.location.values:
        yield ("DG-" in l or "CA" in l)

In [10]:
def hippocampal_sampling(sampling):
    signals = {}
    for probe, signal in sampling.signals.items():
        visual_channels = signal.channels.location[["VIS" in loc for loc in signal.channels.location]]
        visual_area = os.path.commonprefix(list(visual_channels.values))
        hippocampal_signal = signal.select_channels(list(hippocampal_channels(signal.channels)))
        if len(hippocampal_signal.channels) > 0:
            signals[visual_area] = hippocampal_signal
    return sampling.__class__(sampling.intervals, sampling.trials, sampling.units, **signals)

In [11]:
def samplings():
    for cond in CONDITIONS:
        for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
            subject = subject_dir.split('/')[-2]
            if not os.path.exists(subject_dir + cond):
                continue
            sampling = hippocampal_sampling(epych.recording.Sampling.unpickle(subject_dir + cond))
            yield sampling
            del sampling
            logging.info("Loaded LFPs for %s in subject %s" % (cond, subject))
        logging.info("Loaded LFPs for %s" % cond)

In [12]:
def probe_location_key(area, sig):
    return area

In [13]:
alignment = epych.statistics.alignment.AlignmentSummary(signal_key=probe_location_key, alignment=epych.statistics.alignment.subcortical_alignment)

In [14]:
alignment.calculate(samplings())

INFO:root:Loaded LFPs for go_gloexp in subject sub-621890
INFO:root:Loaded LFPs for go_gloexp in subject sub-632485
INFO:root:Loaded LFPs for go_gloexp in subject sub-632487
INFO:root:Loaded LFPs for go_gloexp in subject sub-637542
INFO:root:Loaded LFPs for go_gloexp in subject sub-637908
INFO:root:Loaded LFPs for go_gloexp in subject sub-637909
INFO:root:Loaded LFPs for go_gloexp in subject sub-640507
INFO:root:Loaded LFPs for go_gloexp in subject sub-642507
INFO:root:Loaded LFPs for go_gloexp in subject sub-645322
INFO:root:Loaded LFPs for go_gloexp in subject sub-645324
INFO:root:Loaded LFPs for go_gloexp in subject sub-645495
INFO:root:Loaded LFPs for go_gloexp in subject sub-647836
INFO:root:Loaded LFPs for go_gloexp in subject sub-649323
INFO:root:Loaded LFPs for go_gloexp in subject sub-649324
INFO:root:Loaded LFPs for go_gloexp
INFO:root:Loaded LFPs for go_seqctl in subject sub-621890
INFO:root:Loaded LFPs for go_seqctl in subject sub-632485
INFO:root:Loaded LFPs for go_seqctl 

{'VISam': <epych.statistics.alignment.SubcorticalAlignment at 0x75bffc677d40>,
 'VISpm': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdad1f2330>,
 'VISl': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdac1500e0>,
 'VISrl': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdac152cc0>,
 'VIS': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdac18eea0>,
 'VISal': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdac18f7d0>,
 'VISp': <epych.statistics.alignment.SubcorticalAlignment at 0x75bdac1e61e0>}

In [15]:
alignment.pickle("/mnt/data/000253/hippocampal_alignment")